In [1]:
import os
import sys
#sys.path.insert(0, os.path.dirname(os.getcwd()))
import pickle

import numpy as np
import torch
#import torch.cuda
import timeit
import pandas as pd
#import argparse
import itertools
from load_data import load_data # segment_SHAP
from evaluation.metrics.AUC_difference import AUC_difference
from predictor_utils import load_predictor, predict_proba
from pickle import dump



In [2]:
# PATHS
datasets_folder_path = None
attributions_folder_path = None
trained_models_folder_path = None
save_results_folder_path = None

In [3]:
# settings
dataset_names = ['MP']    #[sys.argv[1]]
predictor_names = ['randomForest']    #[sys.argv[2]] ["randomForest", 'miniRocket', 'resNet', "QUANT"]
segmentation_names = ["clasp","greedygaussian", "equal", "infogain","nnsegment"] # ["clasp","greedygaussian", "equal", "infogain","nnsegment"] 
background_names =  ["average", "zero","sampling"] #["average", "zero", "sampling"]
normalization_names = ["default", "normalized"]

metric_names = ["AUC_difference"]

demo_mode = True
# demo
if demo_mode:
    dataset_names = ['gunpoint']
    predictor_names = ['randomForest', 'resNet', 'miniRocket']
    segmentation_names = ['clasp']
    background_names = ["average", 'zero'] #,'sampling']
    normalization_names = ["default", "normalized"]

# optional
# # get infos about which explanations are evaluated
# datasets = list( explanations['attributions'].keys() )
# segmentations = list( explanations['attributions'][datasets[0]].keys() )
# predictors = list( explanations['attributions'][datasets[0]][segmentations[0]].keys() )
# backgrounds = list( explanations['attributions'][datasets[0]][segmentations[0]][predictors[0]].keys() )
# result_types = ['default','normalized']

In [9]:
cwd = os.getcwd()
if datasets_folder_path is None:
    datasets_folder_path = "datasets" #os.path.join(cwd, "datasets")
if attributions_folder_path is None:
    attributions_folder_path = "attributions"
if trained_models_folder_path is None:
    trained_models_folder_path = "trained_models"
if save_results_folder_path is None:
    save_results_folder_path = os.path.join("evaluation", "results")

# device
device = "cuda" if torch.cuda.is_available() else "cpu"

starttime = timeit.default_timer()

for dataset_name in dataset_names:
    print("Dataset: ", dataset_name)
    # loading dataset
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name, path=datasets_folder_path)
    if demo_mode:
        X_test, y_test = X_test[:2], y_test[:2]


    for predictor_name in predictor_names:
        print("Predictor: ", predictor_name)

        # load classifier
        predictor = load_predictor(path=trained_models_folder_path, predictor_name=predictor_name, dataset_name=dataset_name, device=device) # torch.device(device)

        # load explanations
        attribution_filename = "_".join(("all_results", dataset_name, predictor_name)) + ".npy"
        explanations = np.load(os.path.join(attributions_folder_path, attribution_filename), allow_pickle=True).item() # FileNotFoundError
        label_mapping = explanations['label_mapping'][dataset_name]

        data_list = []

        for key in itertools.product(segmentation_names, background_names, normalization_names, metric_names):
            segmentation_name, background_name, normalization_name, metric_name = key
            print("assessing", key)

            # load model and explanations to access
            try:
                attributions = explanations['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name]
            except KeyError as error:
                print('Warning: attributions is missing keys, skipping to next ' + repr(error))
                continue
            try:
                y_test_pred = explanations['y_test_pred'][dataset_name][predictor_name]
            except KeyError:
                y_test_pred = explanations['y_test_pred'][dataset_name][segmentation_name][predictor_name]

            if metric_name=="AUC_difference":
                results = AUC_difference(predictor, X_train, X_test, y_train, attributions, y_test_pred, label_mapping, n_steps=18)
            else:
                raise ValueError(f"metric name {metric_name} is not supported")
            for result_tuple in results:
                data_list.append((dataset_name, segmentation_name, predictor_name, background_name, normalization_name, metric_name) + result_tuple)

            print("elapsed time", (timeit.default_timer() - starttime))

        # save
        column_names = ['Dataset', 'Segmentation', 'ML model', 'Background', 'Normalization', 'Metric', 'Perturb', "Result"]
        df = pd.DataFrame(data=data_list, columns = column_names)
        file_name = "_".join(("evaluation", predictor_name, dataset_name))
        result_path = os.path.join(save_results_folder_path, file_name)
        if demo_mode:
            df.to_csv(result_path)
            # with open( "_".join( (dataset_name,classifier_name)) ,"wb") as f:
            #     pickle.dump(results_dict, f)



Dataset:  gunpoint
Predictor:  randomForest
assessing ('clasp', 'average', 'default', 'AUC_difference')
elapsed time 2.2444441999541596
assessing ('clasp', 'average', 'normalized', 'AUC_difference')
elapsed time 4.553953999944497
assessing ('clasp', 'zero', 'default', 'AUC_difference')
elapsed time 6.768850999942515
assessing ('clasp', 'zero', 'normalized', 'AUC_difference')
elapsed time 8.961783099977765
Predictor:  resNet
assessing ('clasp', 'average', 'default', 'AUC_difference')
elapsed time 9.482537299976684
assessing ('clasp', 'average', 'normalized', 'AUC_difference')
elapsed time 10.004455199989025
assessing ('clasp', 'zero', 'default', 'AUC_difference')
elapsed time 10.631013299978804
assessing ('clasp', 'zero', 'normalized', 'AUC_difference')
elapsed time 11.130052399996202
Predictor:  miniRocket
assessing ('clasp', 'average', 'default', 'AUC_difference')
elapsed time 11.511035299976356
assessing ('clasp', 'average', 'normalized', 'AUC_difference')
elapsed time 12.18917789997

In [6]:
# column_names = ['Dataset', 'Segmentation', 'ML model', 'Background', 'Normalization', 'Metric', 'Perturb', "Result"]
# df = pd.DataFrame(data=data_list, columns = column_names)

In [7]:
# df

,Dataset,Segmentation,ML model,Background,Normalization,Metric,Perturb,Result
0,gunpoint,clasp,randomForest,average,default,AUC_difference,default,0.230263
1,gunpoint,clasp,randomForest,average,default,AUC_difference,normalized,0.359317
2,gunpoint,clasp,randomForest,average,normalized,AUC_difference,default,0.146579
3,gunpoint,clasp,randomForest,average,normalized,AUC_difference,normalized,0.234831
4,gunpoint,clasp,randomForest,zero,default,AUC_difference,default,0.040000
5,gunpoint,clasp,randomForest,zero,default,AUC_difference,normalized,0.061177
6,gunpoint,clasp,randomForest,zero,normalized,AUC_difference,default,-0.087895
7,gunpoint,clasp,randomForest,zero,normalized,AUC_difference,normalized,-0.134853
8,gunpoint,clasp,resNet,average,default,AUC_difference,default,0.210711
9,gunpoint,clasp,resNet,average,default,AUC_difference,normalized,2.030536


In [ ]:
# df2 = df.pivot(index=["Dataset", "Segmentation", "ML model", "Background", "Normalization", "Perturb", "Metric"], columns="Result", values="Result")
# df2.to_dict(orient="index")